In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import pandas as pd
from IPython.display import HTML
from pandas_summary import DataFrameSummary
from datetime import datetime, timedelta
import math
import numpy as np
from datetime import date
from fastai.tabular.transform import add_datepart

In [3]:
! pwd

/home/paperspace/fastai/courses/dl1_recreate


In [4]:
PATH="/home/paperspace/fastai/courses/dl1_recreate/"

In [5]:
DATA_PATH=f'{PATH}data/rossmann/'

In [6]:
! ls {DATA_PATH}

googletrend.csv        state_names.csv	store_states.csv  train.csv
sample_submission.csv  store.csv	test.csv	  weather.csv


In [7]:
! cat {DATA_PATH}googletrend.csv | head

file,week,trend
Rossmann_DE_SN,2012-12-02 - 2012-12-08,96
Rossmann_DE_SN,2012-12-09 - 2012-12-15,95
Rossmann_DE_SN,2012-12-16 - 2012-12-22,91
Rossmann_DE_SN,2012-12-23 - 2012-12-29,48
Rossmann_DE_SN,2012-12-30 - 2013-01-05,67
Rossmann_DE_SN,2013-01-06 - 2013-01-12,71
Rossmann_DE_SN,2013-01-13 - 2013-01-19,53
Rossmann_DE_SN,2013-01-20 - 2013-01-26,58
Rossmann_DE_SN,2013-01-27 - 2013-02-02,55
cat: write error: Broken pipe


In [8]:
! cat {DATA_PATH}train.csv | wc -l

1017210


In [9]:
! cat {DATA_PATH}test.csv | wc -l

41089


I grabbed the csv's pre-combined from a github repo -- normally the data comes in csv "shards" that have to be combined to create the entire dataset. 

In [10]:
csvs = ['googletrend', 'state_names', 'store_states', 'train', 'store', 'test', 'weather']

In [11]:
dfs = [pd.read_csv(f'{DATA_PATH}{fname}.csv') for fname in csvs]

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3183: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [12]:
for (name, frame) in [(j, i.head()) for (i, j) in zip(dfs, csvs)]: display(name, frame)

'googletrend'

,file,week,trend
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67


'state_names'

,StateName,State
0,BadenWuerttemberg,BW
1,Bayern,BY
2,Berlin,BE
3,Brandenburg,BB
4,Bremen,HB


'store_states'

...

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


'test'

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


'weather'

,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,...,31.0,12.0,4.0,39,26,58.0,5.08,6.0,Rain,215
1,NordrheinWestfalen,2013-01-02,7,4,1,5,3,2,93,85,...,31.0,14.0,10.0,24,16,NaN,0.00,6.0,Rain,225
2,NordrheinWestfalen,2013-01-03,11,8,6,10,8,4,100,93,...,31.0,8.0,2.0,26,21,NaN,1.02,7.0,Rain,240
3,NordrheinWestfalen,2013-01-04,9,9,8,9,9,8,100,94,...,11.0,5.0,2.0,23,14,NaN,0.25,7.0,Rain,263
4,NordrheinWestfalen,2013-01-05,8,8,7,8,7,6,100,94,...,10.0,6.0,3.0,16,10,NaN,0.00,7.0,Rain,268


In [13]:
googletrend, state_names, store_states, train, store, test, weather = dfs

In [14]:
googletrend.dtypes

file     object
week     object
trend     int64
dtype: object

In [15]:
[display(name, i.dtypes) for (name, i) in zip(csvs, dfs)]

'googletrend'

file     object
week     object
trend     int64
dtype: object

'state_names'

StateName    object
State        object
dtype: object

'store_states'

...

'test'

Id                 int64
Store              int64
DayOfWeek          int64
Date              object
Open             float64
Promo              int64
StateHoliday      object
SchoolHoliday      int64
dtype: object

'weather'

file                           object
Date                           object
Max_TemperatureC                int64
Mean_TemperatureC               int64
Min_TemperatureC                int64
Dew_PointC                      int64
MeanDew_PointC                  int64
Min_DewpointC                   int64
Max_Humidity                    int64
Mean_Humidity                   int64
Min_Humidity                    int64
Max_Sea_Level_PressurehPa       int64
Mean_Sea_Level_PressurehPa      int64
Min_Sea_Level_PressurehPa       int64
Max_VisibilityKm              float64
Mean_VisibilityKm             float64
Min_VisibilitykM              float64
Max_Wind_SpeedKm_h              int64
Mean_Wind_SpeedKm_h             int64
Max_Gust_SpeedKm_h            float64
Precipitationmm               float64
CloudCover                    float64
Events                         object
WindDirDegrees                  int64
dtype: object

[None, None, None, None, None, None, None]

In [16]:
train.StateHoliday = train.StateHoliday != '0'

In [17]:
display(train.dtypes)

Store             int64
DayOfWeek         int64
Date             object
Sales             int64
Customers         int64
Open              int64
Promo             int64
StateHoliday       bool
SchoolHoliday     int64
dtype: object

In [18]:
train.Date.head()

0    2015-07-31
1    2015-07-31
2    2015-07-31
3    2015-07-31
4    2015-07-31
Name: Date, dtype: object

In [19]:
googletrend.head()

,file,week,trend
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67


In [20]:
googletrend.file.unique()

array(['Rossmann_DE_SN', 'Rossmann_DE', 'Rossmann_DE_BY', 'Rossmann_DE_TH', 'Rossmann_DE_NW', 'Rossmann_DE_BE',
       'Rossmann_DE_RP', 'Rossmann_DE_BW', 'Rossmann_DE_NI', 'Rossmann_DE_SH', 'Rossmann_DE_HE', 'Rossmann_DE_ST',
       'Rossmann_DE_HH', 'Rossmann_DE_SL'], dtype=object)

In [21]:
googletrend.file = [i.split('_')[-1] for i in googletrend.file]

In [22]:
googletrend.week = [i.split(' - ')[0] for i in googletrend.week]

In [23]:
googletrend.head()

,file,week,trend
0,SN,2012-12-02,96
1,SN,2012-12-09,95
2,SN,2012-12-16,91
3,SN,2012-12-23,48
4,SN,2012-12-30,67


In [24]:
googletrend.columns = ['state', 'week', 'trend']

In [25]:
googletrend.head()

,state,week,trend
0,SN,2012-12-02,96
1,SN,2012-12-09,95
2,SN,2012-12-16,91
3,SN,2012-12-23,48
4,SN,2012-12-30,67


In [26]:
weather.head()

,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,...,31.0,12.0,4.0,39,26,58.0,5.08,6.0,Rain,215
1,NordrheinWestfalen,2013-01-02,7,4,1,5,3,2,93,85,...,31.0,14.0,10.0,24,16,NaN,0.00,6.0,Rain,225
2,NordrheinWestfalen,2013-01-03,11,8,6,10,8,4,100,93,...,31.0,8.0,2.0,26,21,NaN,1.02,7.0,Rain,240
3,NordrheinWestfalen,2013-01-04,9,9,8,9,9,8,100,94,...,11.0,5.0,2.0,23,14,NaN,0.25,7.0,Rain,263
4,NordrheinWestfalen,2013-01-05,8,8,7,8,7,6,100,94,...,10.0,6.0,3.0,16,10,NaN,0.00,7.0,Rain,268


In [27]:
weather.isna().sum(axis=0)

file                              0
Date                              0
Max_TemperatureC                  0
Mean_TemperatureC                 0
Min_TemperatureC                  0
Dew_PointC                        0
MeanDew_PointC                    0
Min_DewpointC                     0
Max_Humidity                      0
Mean_Humidity                     0
Min_Humidity                      0
Max_Sea_Level_PressurehPa         0
Mean_Sea_Level_PressurehPa        0
Min_Sea_Level_PressurehPa         0
Max_VisibilityKm                381
Mean_VisibilityKm               381
Min_VisibilitykM                381
Max_Wind_SpeedKm_h                0
Mean_Wind_SpeedKm_h               0
Max_Gust_SpeedKm_h            12236
Precipitationmm                   0
CloudCover                     1173
Events                         3951
WindDirDegrees                    0
dtype: int64

In [28]:
weather.size

380160

In [29]:
weather.describe()

,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,Min_Humidity,Max_Sea_Level_PressurehPa,...,Min_Sea_Level_PressurehPa,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,WindDirDegrees
count,15840.000000,15840.000000,15840.000000,15840.000000,15840.000000,15840.000000,15840.000000,15840.000000,15840.000000,15840.000000,...,15840.000000,15459.000000,15459.000000,15459.000000,15840.000000,15840.000000,3604.000000,15840.000000,14667.000000,15840.000000
mean,14.644129,10.388952,6.198990,8.587816,6.205808,3.626136,93.659596,74.282891,50.158586,1018.532197,...,1012.307955,24.057572,12.239796,7.025163,22.766604,11.972222,48.864317,0.831718,5.551306,175.896717
std,8.646012,7.379260,6.526391,6.244780,6.086768,6.128390,7.678530,13.486552,19.960216,7.788720,...,8.600585,8.976799,5.067944,4.980602,8.988618,5.872844,13.026954,2.513506,1.687710,101.588872
min,-11.000000,-13.000000,-15.000000,-14.000000,-15.000000,-73.000000,44.000000,30.000000,4.000000,976.000000,...,970.000000,0.000000,0.000000,0.000000,3.000000,2.000000,21.000000,0.000000,0.000000,-1.000000
25%,8.000000,4.000000,1.000000,4.000000,2.000000,-1.000000,90.750000,65.000000,34.000000,1014.000000,...,1007.000000,14.000000,10.000000,3.000000,16.000000,8.000000,39.000000,0.000000,5.000000,80.000000
50%,15.000000,11.000000,7.000000,9.000000,7.000000,4.000000,94.000000,76.000000,49.000000,1019.000000,...,1013.000000,31.000000,11.000000,7.000000,21.000000,11.000000,48.000000,0.000000,6.000000,202.000000
75%,21.000000,16.000000,11.000000,13.000000,11.000000,8.000000,100.000000,85.000000,66.000000,1024.000000,...,1018.000000,31.000000,14.000000,10.000000,27.000000,14.000000,55.000000,0.250000,7.000000,256.000000
max,39.000000,31.000000,24.000000,25.000000,20.000000,19.000000,100.000000,100.000000,100.000000,1043.000000,...,1038.000000,31.000000,31.000000,31.000000,101.000000,53.000000,111.000000,58.930000,8.000000,360.000000


In [30]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,False,1
1,2,5,2015-07-31,6064,625,1,1,False,1
2,3,5,2015-07-31,8314,821,1,1,False,1
3,4,5,2015-07-31,13995,1498,1,1,False,1
4,5,5,2015-07-31,4822,559,1,1,False,1


In [31]:
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [37]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,False,1
1,2,5,2015-07-31,6064,625,1,1,False,1
2,3,5,2015-07-31,8314,821,1,1,False,1
3,4,5,2015-07-31,13995,1498,1,1,False,1
4,5,5,2015-07-31,4822,559,1,1,False,1


# Impute Missing Values, Compute Some Durations

In [47]:
avg_days_in_month = 30.42

In [48]:
store['Promo2SinceMonth'] = np.int64(np.ceil(store['Promo2SinceWeek'].astype(np.float64) * 7 / avg_days_in_month))

In [65]:
store['CompetitionDistance'].fillna(0.0, inplace=True)
store['CompetitionOpenSinceMonth'].fillna(1, inplace=True)
store['CompetitionOpenSinceYear'].fillna(1900, inplace=True)
store['PromoInterval'].fillna('', inplace=True)
store['Promo2SinceWeek'].fillna(1, inplace=True)
store['Promo2SinceMonth'].fillna(1, inplace=True)
store['Promo2SinceYear'].fillna(1900, inplace=True)

In [66]:
store['Promo2Since'] = store.apply(lambda x: datetime(year=int(x['Promo2SinceYear']), month=int(x['Promo2SinceMonth']), day=15), axis=1)

In [68]:
train_store = train.merge(store, on='Store', suffixes=['', '_y'])

In [69]:
train_store['CompetitionOpenSince'] = train_store.apply(lambda x: datetime(year=int(x['CompetitionOpenSinceYear']), month=int(x['CompetitionOpenSinceMonth']), day=15), axis=1)

In [70]:
train_store['CompetitionOpenDays'] = np.floor(train_store.apply(lambda x: (x['Date'] - x['CompetitionOpenSince']) / timedelta(days=1), axis=1))

In [71]:
train_store['CompetitionOpenSinceWeeks'] = np.floor(train_store['CompetitionOpenDays'] / 7.0)

In [72]:
train_store['CompetitionOpenSinceMonths'] = np.floor(train_store['CompetitionOpenDays'] / avg_days_in_month)

In [75]:
train_store['Promo2SinceDays'] = np.floor(train_store.apply(lambda x: (x['Date'] - x['Promo2Since']) / timedelta(days=1), axis=1))

In [76]:
train_store['Promo2SinceWeeks'] = np.floor(np.ceil(train_store['Promo2SinceDays'] / 7.0))

In [77]:
train_store['Promo2SinceMonths'] = np.floor(np.ceil(train_store['Promo2SinceDays'] / avg_days_in_month))

In [78]:
train['Date'] = train['Date'].astype('datetime64[D]')

In [79]:
weather.isna().sum(axis=0)

file                          0
Date                          0
Max_TemperatureC              0
Mean_TemperatureC             0
Min_TemperatureC              0
Dew_PointC                    0
MeanDew_PointC                0
Min_DewpointC                 0
Max_Humidity                  0
Mean_Humidity                 0
Min_Humidity                  0
Max_Sea_Level_PressurehPa     0
Mean_Sea_Level_PressurehPa    0
Min_Sea_Level_PressurehPa     0
Max_VisibilityKm              0
Mean_VisibilityKm             0
Min_VisibilitykM              0
Max_Wind_SpeedKm_h            0
Mean_Wind_SpeedKm_h           0
Max_Gust_SpeedKm_h            0
Precipitationmm               0
CloudCover                    0
Events                        0
WindDirDegrees                0
dtype: int64

In [80]:
weather['Max_VisibilityKm'].fillna(0.0, inplace=True)
weather['Mean_VisibilityKm'].fillna(0.0, inplace=True)
weather['Min_VisibilitykM'].fillna(0.0, inplace=True)
weather['Max_Gust_SpeedKm_h'].fillna(0.0, inplace=True)
weather['CloudCover'].fillna(0.0, inplace=True)
weather['Events'].fillna(0.0, inplace=True)

In [81]:
train_store_dfs = DataFrameSummary(store); train_store_dfs.columns_stats

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Promo2SinceMonth,Promo2Since
counts,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115
uniques,1115,4,3,655,12,23,2,24,8,4,12,52
missing,0,0,0,0,0,0,0,0,0,0,0,0
missing_perc,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
types,numeric,categorical,categorical,numeric,numeric,numeric,bool,numeric,numeric,categorical,numeric,date


In [82]:
train_store.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,PromoInterval,Promo2SinceMonth,Promo2Since,CompetitionOpenSince,CompetitionOpenDays,CompetitionOpenSinceWeeks,CompetitionOpenSinceMonths,Promo2SinceDays,Promo2SinceWeeks,Promo2SinceMonths
0,1,5,2015-07-31,5263,555,1,1,False,1,c,...,,1,1900-01-15,2008-09-15,2510.0,358.0,82.0,42200.0,6029.0,1388.0
1,1,4,2015-07-30,5020,546,1,1,False,1,c,...,,1,1900-01-15,2008-09-15,2509.0,358.0,82.0,42199.0,6029.0,1388.0
2,1,3,2015-07-29,4782,523,1,1,False,1,c,...,,1,1900-01-15,2008-09-15,2508.0,358.0,82.0,42198.0,6029.0,1388.0
3,1,2,2015-07-28,5011,560,1,1,False,1,c,...,,1,1900-01-15,2008-09-15,2507.0,358.0,82.0,42197.0,6029.0,1388.0
4,1,1,2015-07-27,6102,612,1,1,False,1,c,...,,1,1900-01-15,2008-09-15,2506.0,358.0,82.0,42196.0,6028.0,1388.0


In [62]:
store_states.head()

,Store,State
0,1,HE
1,2,TH
2,3,NW
3,4,BE
4,5,SN


In [63]:
weather.head()

,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,...,31.0,12.0,4.0,39,26,58.0,5.08,6.0,Rain,215
1,NordrheinWestfalen,2013-01-02,7,4,1,5,3,2,93,85,...,31.0,14.0,10.0,24,16,NaN,0.00,6.0,Rain,225
2,NordrheinWestfalen,2013-01-03,11,8,6,10,8,4,100,93,...,31.0,8.0,2.0,26,21,NaN,1.02,7.0,Rain,240
3,NordrheinWestfalen,2013-01-04,9,9,8,9,9,8,100,94,...,11.0,5.0,2.0,23,14,NaN,0.25,7.0,Rain,263
4,NordrheinWestfalen,2013-01-05,8,8,7,8,7,6,100,94,...,10.0,6.0,3.0,16,10,NaN,0.00,7.0,Rain,268


In [84]:
weather_state_names = weather.merge(state_names, how='left', left_on='file', right_on='StateName', suffixes=['', '_y'])

In [85]:
weather_state_names.head()

,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees,StateName,State
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,...,4.0,39,26,58.0,5.08,6.0,Rain,215,NordrheinWestfalen,NW
1,NordrheinWestfalen,2013-01-02,7,4,1,5,3,2,93,85,...,10.0,24,16,0.0,0.00,6.0,Rain,225,NordrheinWestfalen,NW
2,NordrheinWestfalen,2013-01-03,11,8,6,10,8,4,100,93,...,2.0,26,21,0.0,1.02,7.0,Rain,240,NordrheinWestfalen,NW
3,NordrheinWestfalen,2013-01-04,9,9,8,9,9,8,100,94,...,2.0,23,14,0.0,0.25,7.0,Rain,263,NordrheinWestfalen,NW
4,NordrheinWestfalen,2013-01-05,8,8,7,8,7,6,100,94,...,3.0,16,10,0.0,0.00,7.0,Rain,268,NordrheinWestfalen,NW


In [86]:
weather_state_names.drop('file', inplace=True, axis=1)

In [87]:
weather_state_names.drop('StateName', inplace=True, axis=1)

In [88]:
weather_state_names.head()

,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,Min_Humidity,...,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees,State
0,2013-01-01,8,4,2,7,5,1,94,87,65,...,12.0,4.0,39,26,58.0,5.08,6.0,Rain,215,NW
1,2013-01-02,7,4,1,5,3,2,93,85,78,...,14.0,10.0,24,16,0.0,0.00,6.0,Rain,225,NW
2,2013-01-03,11,8,6,10,8,4,100,93,77,...,8.0,2.0,26,21,0.0,1.02,7.0,Rain,240,NW
3,2013-01-04,9,9,8,9,9,8,100,94,87,...,5.0,2.0,23,14,0.0,0.25,7.0,Rain,263,NW
4,2013-01-05,8,8,7,8,7,6,100,94,87,...,6.0,3.0,16,10,0.0,0.00,7.0,Rain,268,NW


In [89]:
wsn_summary = DataFrameSummary(weather_state_names)

In [90]:
wsn_summary.columns_stats

,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,Min_Humidity,...,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees,State
counts,15840,15840,15840,15840,15840,15840,15840,15840,15840,15840,...,15840,15840,15840,15840,15840,15840,15840,15840,15840,15840
uniques,990,51,45,40,40,36,40,53,71,93,...,32,24,44,29,48,41,9,22,362,16
missing,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
missing_perc,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,...,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
types,categorical,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,...,numeric,numeric,numeric,numeric,numeric,numeric,numeric,categorical,numeric,categorical


In [91]:
weather_state_names.describe()

,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,Min_Humidity,Max_Sea_Level_PressurehPa,...,Min_Sea_Level_PressurehPa,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,WindDirDegrees
count,15840.000000,15840.000000,15840.000000,15840.000000,15840.000000,15840.000000,15840.000000,15840.000000,15840.000000,15840.000000,...,15840.000000,15840.000000,15840.000000,15840.000000,15840.000000,15840.000000,15840.000000,15840.000000,15840.000000,15840.000000
mean,14.644129,10.388952,6.198990,8.587816,6.205808,3.626136,93.659596,74.282891,50.158586,1018.532197,...,1012.307955,23.478914,11.945391,6.856187,22.766604,11.972222,11.117866,0.831718,5.140215,175.896717
std,8.646012,7.379260,6.526391,6.244780,6.086768,6.128390,7.678530,13.486552,19.960216,7.788720,...,8.600585,9.603731,5.346329,5.036695,8.988618,5.872844,21.407700,2.513506,2.179599,101.588872
min,-11.000000,-13.000000,-15.000000,-14.000000,-15.000000,-73.000000,44.000000,30.000000,4.000000,976.000000,...,970.000000,0.000000,0.000000,0.000000,3.000000,2.000000,0.000000,0.000000,0.000000,-1.000000
25%,8.000000,4.000000,1.000000,4.000000,2.000000,-1.000000,90.750000,65.000000,34.000000,1014.000000,...,1007.000000,13.000000,9.000000,3.000000,16.000000,8.000000,0.000000,0.000000,4.000000,80.000000
50%,15.000000,11.000000,7.000000,9.000000,7.000000,4.000000,94.000000,76.000000,49.000000,1019.000000,...,1013.000000,31.000000,11.000000,6.000000,21.000000,11.000000,0.000000,0.000000,6.000000,202.000000
75%,21.000000,16.000000,11.000000,13.000000,11.000000,8.000000,100.000000,85.000000,66.000000,1024.000000,...,1018.000000,31.000000,14.000000,10.000000,27.000000,14.000000,0.000000,0.250000,7.000000,256.000000
max,39.000000,31.000000,24.000000,25.000000,20.000000,19.000000,100.000000,100.000000,100.000000,1043.000000,...,1038.000000,31.000000,31.000000,31.000000,101.000000,53.000000,111.000000,58.930000,8.000000,360.000000


In [92]:
weather_state_names.head()

,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,Min_Humidity,...,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees,State
0,2013-01-01,8,4,2,7,5,1,94,87,65,...,12.0,4.0,39,26,58.0,5.08,6.0,Rain,215,NW
1,2013-01-02,7,4,1,5,3,2,93,85,78,...,14.0,10.0,24,16,0.0,0.00,6.0,Rain,225,NW
2,2013-01-03,11,8,6,10,8,4,100,93,77,...,8.0,2.0,26,21,0.0,1.02,7.0,Rain,240,NW
3,2013-01-04,9,9,8,9,9,8,100,94,87,...,5.0,2.0,23,14,0.0,0.25,7.0,Rain,263,NW
4,2013-01-05,8,8,7,8,7,6,100,94,87,...,6.0,3.0,16,10,0.0,0.00,7.0,Rain,268,NW


In [93]:
# weather_state_names.to_feather('weather_state_names')

In [94]:
train_store.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,PromoInterval,Promo2SinceMonth,Promo2Since,CompetitionOpenSince,CompetitionOpenDays,CompetitionOpenSinceWeeks,CompetitionOpenSinceMonths,Promo2SinceDays,Promo2SinceWeeks,Promo2SinceMonths
0,1,5,2015-07-31,5263,555,1,1,False,1,c,...,,1,1900-01-15,2008-09-15,2510.0,358.0,82.0,42200.0,6029.0,1388.0
1,1,4,2015-07-30,5020,546,1,1,False,1,c,...,,1,1900-01-15,2008-09-15,2509.0,358.0,82.0,42199.0,6029.0,1388.0
2,1,3,2015-07-29,4782,523,1,1,False,1,c,...,,1,1900-01-15,2008-09-15,2508.0,358.0,82.0,42198.0,6029.0,1388.0
3,1,2,2015-07-28,5011,560,1,1,False,1,c,...,,1,1900-01-15,2008-09-15,2507.0,358.0,82.0,42197.0,6029.0,1388.0
4,1,1,2015-07-27,6102,612,1,1,False,1,c,...,,1,1900-01-15,2008-09-15,2506.0,358.0,82.0,42196.0,6028.0,1388.0


In [95]:
store_states.head()

,Store,State
0,1,HE
1,2,TH
2,3,NW
3,4,BE
4,5,SN


In [96]:
train_store_state = train_store.merge(store_states, how='left', on='Store', suffixes=['', '_y'])

In [97]:
weather_state_names['Date'] = weather_state_names['Date'].astype('datetime64[D]')

In [196]:
train_store_state_weather = train_store_state.merge(weather_state_names, how='left', on=['State', 'Date'], suffixes=['', '_y'])

In [197]:
train_store_state_weather.drop('Promo2Since', inplace=True, axis=1)
train_store_state_weather.drop('CompetitionOpenSince', inplace=True, axis=1)

In [198]:
train_store_state_weather.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees
0,1,5,2015-07-31,5263,555,1,1,False,1,c,...,31.0,15.0,10.0,24,11,0.0,0.00,1.0,Fog,13
1,1,4,2015-07-30,5020,546,1,1,False,1,c,...,10.0,10.0,10.0,29,11,53.0,0.00,4.0,Rain,253
2,1,3,2015-07-29,4782,523,1,1,False,1,c,...,31.0,11.0,10.0,32,21,42.0,0.00,5.0,Rain,231
3,1,2,2015-07-28,5011,560,1,1,False,1,c,...,14.0,10.0,10.0,40,23,55.0,0.00,5.0,Rain,236
4,1,1,2015-07-27,6102,612,1,1,False,1,c,...,31.0,12.0,10.0,50,21,63.0,2.03,6.0,Rain,211


In [199]:
add_datepart(train_store_state_weather, 'Date', False, False)

In [200]:
train_store_state_weather.isna().sum(axis=0)

Store                         0
DayOfWeek                     0
Date                          0
Sales                         0
Customers                     0
Open                          0
Promo                         0
StateHoliday                  0
SchoolHoliday                 0
StoreType                     0
Assortment                    0
CompetitionDistance           0
CompetitionOpenSinceMonth     0
CompetitionOpenSinceYear      0
Promo2                        0
Promo2SinceWeek               0
Promo2SinceYear               0
PromoInterval                 0
Promo2SinceMonth              0
CompetitionOpenDays           0
CompetitionOpenSinceWeeks     0
CompetitionOpenSinceMonths    0
Promo2SinceDays               0
Promo2SinceWeeks              0
Promo2SinceMonths             0
State                         0
Max_TemperatureC              0
Mean_TemperatureC             0
Min_TemperatureC              0
Dew_PointC                    0
                             ..
Min_Dewp

Todo:
* Identify columns that involve "since" and "until", and write some logic that does that
* 7-day rolling averages in both directions for a handful of columns

In [201]:
# rename for clarity
base_df = train_store_state_weather

In [202]:
def get_elapsed(df, date_col, change_col, suffix):
    last_store = -1
    last_date = np.datetime64('NaT')
    vals = []
    for (date, store, col) in zip(df[date_col], df['Store'], df[change_col]):
        if store != last_store: 
            last_date = np.datetime64('NaT')
            last_store = store
        if col:
            last_date = date
        vals.append((date - last_date) / timedelta(days=1))
    df[change_col + suffix] = vals
    return df

In [203]:
a = pd.Series(pd.date_range(pd.datetime.today(), periods=13)).astype('datetime64[D]')

In [204]:
b = [False] * 4 + [True] + [False] * 5 + [False] + [True, False]

In [205]:
b

[False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False]

In [206]:
c = [1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3]

In [207]:
test_df = pd.DataFrame({'a': a, 'b': b, 'Store': c})

In [208]:
get_elapsed(test_df, 'a', 'b', 'After')

,a,b,Store,bAfter
0,2019-01-07,False,1,737065.0
1,2019-01-08,False,1,737066.0
2,2019-01-09,False,2,737067.0
3,2019-01-10,False,2,737068.0
4,2019-01-11,True,2,0.0
5,2019-01-12,False,2,1.0
6,2019-01-13,False,2,2.0
7,2019-01-14,False,2,3.0
8,2019-01-15,False,2,4.0
9,2019-01-16,False,2,5.0


In [213]:
base_df.sort_values(by=['Store', 'Date'], ascending=[True, True], inplace=True)

In [214]:
base_df = get_elapsed(base_df, 'Date', 'StateHoliday', 'After')

In [215]:
base_df.sort_values(by=['Date', 'Store'], ascending=[True, False], inplace=True)

In [217]:
base_df = get_elapsed(base_df, 'Date', 'StateHoliday', 'Before')

In [218]:
base_df.sort_values(by=['Store', 'Date'], ascending=[True, True], inplace=True)

In [234]:
tmp_df = base_df.set_index('Date')

In [235]:
columns = ['SchoolHoliday', 'StateHoliday', 'Promo'] 

In [261]:
fwd = tmp_df[['Store'] + columns].sort_index().groupby('Store').rolling(7, min_periods=1).sum()
bwd = tmp_df[['Store'] + columns].sort_index(ascending=False).groupby('Store').rolling(7, min_periods=1).sum()

In [263]:
fwd = fwd.drop('Store', axis=1).reset_index()
bwd = bwd.drop('Store', axis=1).reset_index()

In [264]:
fwd.head()

,Store,Date,SchoolHoliday,StateHoliday,Promo
0,1,2013-01-01,1.0,1.0,0.0
1,1,2013-01-02,2.0,1.0,0.0
2,1,2013-01-03,3.0,1.0,0.0
3,1,2013-01-04,4.0,1.0,0.0
4,1,2013-01-05,5.0,1.0,0.0


In [265]:
bwd.head()

,Store,Date,SchoolHoliday,StateHoliday,Promo
0,1,2015-07-31,1.0,0.0,1.0
1,1,2015-07-30,2.0,0.0,2.0
2,1,2015-07-29,3.0,0.0,3.0
3,1,2015-07-28,4.0,0.0,4.0
4,1,2015-07-27,5.0,0.0,5.0


In [267]:
base_df = base_df.merge(fwd, on=['Store', 'Date'], how='left', suffixes=['', '_fwd'])

In [268]:
base_df = base_df.merge(bwd, on=['Store', 'Date'], how='left', suffixes=['', '_bwd'])

In [270]:
base_df.drop('Date', axis=1, inplace=True)

In [271]:
base_df.drop('rolling_sales', axis=1, inplace=True)

In [278]:
base_df.isna().sum().sum()

0

In [279]:
base_df.columns

Index(['Store', 'DayOfWeek', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval', 'Promo2SinceMonth',
       'CompetitionOpenDays', 'CompetitionOpenSinceWeeks',
       'CompetitionOpenSinceMonths', 'Promo2SinceDays', 'Promo2SinceWeeks',
       'Promo2SinceMonths', 'State', 'Max_TemperatureC', 'Mean_TemperatureC',
       'Min_TemperatureC', 'Dew_PointC', 'MeanDew_PointC', 'Min_DewpointC',
       'Max_Humidity', 'Mean_Humidity', 'Min_Humidity',
       'Max_Sea_Level_PressurehPa', 'Mean_Sea_Level_PressurehPa',
       'Min_Sea_Level_PressurehPa', 'Max_VisibilityKm', 'Mean_VisibilityKm',
       'Min_VisibilitykM', 'Max_Wind_SpeedKm_h', 'Mean_Wind_SpeedKm_h',
       'Max_Gust_SpeedKm_h', 'Precipitationmm', 'CloudCover', 'Events',
       'WindDirDegrees', 'Year', 'Month', 'Week

* Look up heuristic for max number of categories
* Look thru again for any cols that should have been removed
* Split into cont and and cat and do whatever preproc needs to be done
* Create a fastai tabular model